## 前処理 (202113_CSVの中のcsvファイルを、ソートした上で一つにまとめ、total2.csvというファイルに書き出す)

## ライブラリをimportする

In [1]:
import pandas as pd
import glob
import numpy as np

## 202113_CSVの中のcsvファイルを、total.csvという一つのファイルにまとめる

In [2]:
csv_files = glob.glob('NOV_CSV/*.csv')

#csvファイルの中身を追加していくリストを用意
data_list = []

#読み込むファイルのリストを走査
for file in csv_files:
	x = pd.read_csv(file, encoding="shift-jis")
	if len(x.columns) == 14:
		data_list.append(pd.read_csv(file, encoding="shift-jis"))

#リストを全て行方向に結合
#axis=0:行方向に結合, sort
df = pd.concat(data_list, axis=0)

rm_list = ["野菜総量", "その他の菜類", "その他の野菜",
           "輸入野菜計", "他の輸入野菜", "果実総量", "国産果実総量", 
           "その他雑かん", "りんご計", "その他りんご", "日本なし計",
           "その他なし", "かき計", "ぶどう計", "その他ぶどう", "メロン計",
           "その他メロン", "他の国産果実", "輸入果実計", "他の輸入果実"]

for n in rm_list:
	df = df[df["品目名"] != n]

df.to_csv("NOV_TOTAL_CSV/nov_total.csv", index=False)


## total.csvの中身をソートし、total2.csvとして書き出す

In [5]:
df = pd.read_csv('NOV_TOTAL_CSV/nov_total.csv')
df = df.loc[:, ["年", "月", "日", "品目コード", "品目名", "都市コード", "都市名", "産地名", "価格"]]
# df = df.rename(columns={'対前日比（価格)':'価格'})
df = df.fillna('missing')
df = df[df["産地名"] == 'missing']
df = df.drop('産地名', axis=1)
df = df.dropna(how='any', axis=0)
df = df[df["品目名"] != "　　うち輸入"]
# df = df[df["対前日比（価格）"] != '−']
df = df[df["価格"] != '−']
df['key'] = df["日"] + 1000*df["月"] + 1000000*df["年"] + 1000000000*df['都市コード'] + 1000000000000*df['品目コード']
df = df.sort_values(by="key")
df = df.drop('key', axis=1)
df.loc[df['都市コード'] == 401, '都市コード'] = '0401'
df.to_csv("NOV_TOTAL_CSV/nov_total2.csv", index=False)


/Users/sawanorei/opt/anaconda3/envs/titanic/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df = pd.read_csv('NOV_TOTAL_CSV/nov_total.csv')
df = df.loc[:, ["年", "月", "日", "品目コード", "品目名", "都市コード", "都市名", "産地名", "価格"]]
# df = df.rename(columns={'対前日比（価格)':'価格'})
df = df.fillna('missing')
df = df[df["産地名"] == 'missing']
df = df.drop('産地名', axis=1)
df = df.dropna(how='any', axis=0)
df = df[df["品目名"] != "　　うち輸入"]
# df = df[df["対前日比（価格）"] != '−']
df = df[df["価格"] != '−']
df = df[df["価格"] != '…']
df['key'] = df["都市コード"] + 100000*df["日"] + 100000000*df["月"] + \
    100000000000*df['年'] + 100000000000000*df['品目コード']
df = df.sort_values(by="key")
df = df.drop('key', axis=1)
df.loc[df['都市コード'] == 401, '都市コード'] = '0401'

i = 0
for index, data in df.iterrows():
    df.at[df.index[i], '価格'] = str(df.at[df.index[i], '価格']) + '.0'
    i += 1
df.to_csv("NOV_TOTAL_CSV/nov_total3.csv", index=False)


/Users/sawanorei/opt/anaconda3/envs/titanic/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 日毎の平均を算出する

In [8]:
df = pd.read_csv('NOV_TOTAL_CSV/nov_total3.csv')

product = df.at[df.index[0], '品目名']

sum = 0
sum_list = []
num = 0
day = df.at[df.index[0], '日']
month = df.at[df.index[0], '月']
year = df.at[df.index[0], '年']

df['key'] = 0

for index, data in df.iterrows():
	if product != data["品目名"] or day != data["日"] or month != data["月"] or year != data["年"]:
		product = data["品目名"]
		day = data["日"]
		month = data["月"]
		year = data["年"]
		sum_list.append(sum/num)
		sum = 0
		num = 0

	# print(float(data["価格"]))
	# print(data["価格"])
	num += 1
	sum += float(data["価格"])

sum_list.append(sum/num)

product = 0
num = 0
day = 0
month = 0
year = 0

for index, data in df.iterrows():
	if product != data["品目名"] or day != data["日"] or month != data["月"] or year != data["年"]:
		product = data["品目名"]
		day = data["日"]
		month = data["月"]
		year = data["年"]
		df.at[df.index[index], '価格'] = sum_list[num]
		df.at[df.index[index], 'key'] = 1
		num += 1

for index, data in df.iterrows():
	df.at[df.index[index], '文字数'] = int(len(df.at[df.index[index], '品目名']))

df = df[df["key"] == 1]

df = df.drop('key', axis=1)

df = df.drop('都市名', axis=1)

df = df.drop('都市コード', axis=1)

df.to_csv("NOV_TOTAL_CSV/nov_total4.csv", index=False)


## 1週間の平均を算出し、最終日との比を計算する

In [17]:
df = pd.read_csv('NOV_TOTAL_CSV/nov_total4.csv')

product = df.at[df.index[0], '品目名']

sum = 0
sum_list = []
num = 0

df['key'] = -100000*df["日"] - 100000000*df["月"] - \
    100000000000*df['年'] + 100000000000000*df['品目コード']

df = df.sort_values(by="key")

#print(df)

df['key2'] = 0

for index, data in df.iterrows():
	if product != data["品目名"]:
		product = data["品目名"]
		sum_list.append(sum/num)
		sum = 0
		num = 0

	num += 1
	sum += float(data["価格"])

sum_list.append(sum/num)

product = 0
num = 0
i = 0

for index, data in df.iterrows():
	if product != data["品目名"]:
		product = data["品目名"]
		df.at[df.index[i], '価格'] = df.at[df.index[i], '価格']/sum_list[num]
		df.at[df.index[i], 'key2'] = 1
		num += 1
	i += 1

df = df[df["key2"] == 1]

df = df.drop('key', axis=1)
df = df.drop('key2', axis=1)

df.to_csv("NOV_TOTAL_CSV/nov_total5.csv", index=False)

# df.to_csv("total3.csv", index=False)


## 食材のリスト作る

In [18]:
df = pd.read_csv('NOV_TOTAL_CSV/nov_total5.csv')

df['value'] = df['品目コード']
df['label'] = df['品目名']

df = df.drop('年', axis=1)
df = df.drop('月', axis=1)
df = df.drop('価格', axis=1)
df = df.drop('文字数', axis=1)
df = df.drop('日', axis=1)
df = df.drop('品目コード', axis=1)
df = df.drop('品目名', axis=1)

df.to_csv("NOV_TOTAL_CSV/nov_total6.csv", index=False)


## 食材の種類を指定する

In [26]:
df1 = pd.read_csv('NOV_TOTAL_CSV/nov_total5.csv')
df2 = pd.read_csv('NOV_TOTAL_CSV/class.csv')

df1["種類"] = df2["class"]

# for index, data in df.iterrows():
# 	df1.at[df1.index[index], '種類'] = df2.at[df2.index[index], 'class']

df1.to_csv("NOV_TOTAL_CSV/nov_total5.csv", index=False)
